In [ ]:
using Turing
using Random
using Statistics
using Distributions
using StatsPlots
using LaTeXStrings
using DataFrames

Recommended settings for figures.  These are chosen such that if the figure is included into a LaTeX document with approximately half the page width, the annotations will be readable when printed.


In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfontsize=12, lw=2, ms=8)

# Compare Frequentist and Bayesian Interpretations
Following Example 2.55 from the text, we generate $n$ samples from the distribution $N(\theta,1)$, and then try to infer $\theta$ from the data.

In [ ]:
n = 100; # experiment with different values
θₜ = 2.0; # true value

Random.seed!(100); # set a seed for reproducibility

data = rand(Normal(θₜ, 1),n);

## Frequentist Interpretation
In the frequentist interpretation, the only data we can extract is a point estimate of the truth from a simple average:
$$
\hat{\theta} = \frac{1}{n}\sum_{i=1}^n X_i
$$

In [ ]:
@show θ̂ = mean(data);

## Bayesian Interpretation
To apply the Bayesian interpretation, we need to introduce a prior distribution, $\pi_0$, as to what values $\theta$ can take, then since the likelihood is
$$
L(y|\theta) = \exp\left\{-\sum_{i=1}^n\frac{1}{2}|X_i-\theta|^2\right\},
$$
the posterior is
$$
\pi(\theta|y) \propto  \exp\left\{-\sum_{i=1}^n\frac{1}{2}|X_i-\theta|^2\right\} \pi_0(\theta)
$$

In the following, we will take $\pi_0 = N(0,1)$ (try other distributions), and then explore the posterior using the `Turing.jl` module.  For convenience, we use the `HMC` sampler which is generically pretty efficient.

The `Turing.jl` framework has us formulate the problem in a _generative_ way.  Given $\theta$, since the samples are independent, we know that each piece of data should satisfyt $X_i \sim N(\theta, 1)$:

In [ ]:
# define the model
@model function bayes_example(y)
    # this specifis the prior
    θ ~ Normal(0,1);
    # θ ~ Exponential(.1);
    
    n = length(y);
    for i in 1:n
        # each piece of data would satisfy this distribution conditional on θ
        y[i]~ Normal(θ, 1) 
    end
    
end

The module is quite rich with many features, not all of which are illustrated here.  See https://turing.ml/dev/ for additional details.

This samples the posterior distirbution using the HMC algorithm.  This choooses a "step size" of 0.1 and an integration time of 10, generating 10^4 samples.  

In [ ]:
chain = sample(bayes_example(data), HMC(0.1, 10), 10^4)

We can call plot on the `chain` data structure to see both the time series and an estimated density.  Notice how the time series seems to be oscillating about some sort of well characterized mean – this is what we want to see to ensure we are doing a good job of sampling the posterior.

In [ ]:
plot(chain)

In [ ]:
summarize(chain)

In [ ]:
histogram(chain)

The default formatting can be overriden in the standard Julia Plots.jl way:

In [ ]:
histogram(chain, norm=:pdf)
histogram!(bottom_margin=3*StatsPlots.mm)
title!("")
xlabel!(L"$\theta$")
ylabel!("Probability")

Compare posterior with the prior.  First we sample the prior, then visualize.

In [ ]:
chain_prior = sample(bayes_example(data), Prior(), 10^4);


In [ ]:
histogram(chain, norm=:pdf)
histogram!(chain_prior, norm=:pdf)
histogram!(bottom_margin=3*StatsPlots.mm)
title!("")
xlabel!(L"$\theta$")
ylabel!("Probability")

## Accessing the Samples
The generated samples from the posterior for $\theta$ can be obtained by first converting to a data frame and then extracting that column from the data frame.

In [ ]:
df = DataFrame(chain); # convert to a data frame
posterior_samples = df[!,:θ]; # extract the column for θ

In [ ]:
posterior_samples

In [ ]:
df

In [ ]:
@show mean(posterior_samples);
@show var(posterior_samples);

In [ ]:
df_prior = DataFrame(chain_prior); # convert to a data frame
prior_samples = df_prior[!,:θ]; # extract the column for θ

In [ ]:
histogram(posterior_samples, norm=:pdf,label="Posterior", legend=:topleft)
histogram!(prior_samples, norm=:pdf,label="Prior")
xlabel!(L"$\theta$")
ylabel!("Probability")